$\textbf{PROGRAMMING #4 ASSIGNMENT}$
---
         
1. Read the Bernoulli Mixture Model Derivation.
2. Read about Stochastic Expectation-Maximization (EM) Algorithm: https://www.sciencedirect.com/science/article/pii/S0167947320302504.
3. From the given code, modify the EM algorithm to become a Stochastic EM Algorithm.
4. Use the data from the paper: https://www.sciencedirect.com/science/article/abs/pii/S0031320322001753
5. Perform categorical clustering using the Bernoulli Mixture Model with Stochastic EM Algorithm.
6. Compare its performance with K-Modes Algorithm using Folkes-Mallows Index, Adjusted Rand Index, and Normalized Mutual Information Score.
7. Compare and contrast the performances, and explain what is happening (i.e. why is FMI always higher than ARI and NMI? Why is ARI and NMI low compared to FMI? etc.)
8. Write the report in Latex, push to your github with the codes.

Initial Configuration

In [69]:
# Import necessary libraries
import itertools
import networkx as nx
import numpy as np
import pandas as pd
import random
import requests
from kmodes.kmodes import KModes
from sklearn.metrics import adjusted_rand_score as ARI, normalized_mutual_info_score as NMI, fowlkes_mallows_score as FMI
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings

In [2]:
# Ignore all warnings
warnings.filterwarnings('ignore')

In [3]:
# Set the true number of cluster for each dataset
num_clusters_dict = {
    "Soybean": 4,
    "Zoo": 7,
    "Heart Disease": 2,
    "Breast Cancer": 2,
    "Dermatology": 6,
    "Letters(E, F)": 2,
    "DNA": 3,
    "Mushroom": 2,
    "Iris": 3,
    "Isolet": 26,
    "Optical": 10,
    "PenDigits": 10
}

In [8]:
# List of datasets for clustering ensemble task
datasets_ensemble = [
    "Iris",
    "Isolet",
    "Optical",
    "PenDigits"
]

In [9]:
# Set the number of runs for benchmarking
num_runs = 10

In [10]:
# # Dictionary to hold raw dataframes
raw_dataframes = {}

In [11]:
raw_dataframes["Soybean"] = pd.read_csv("https://archive.ics.uci.edu/static/public/91/data.csv")

In [12]:
raw_dataframes["Zoo"] = pd.read_csv("https://archive.ics.uci.edu/static/public/111/data.csv")

In [13]:
raw_dataframes["Heart Disease"] = pd.read_csv("https://archive.ics.uci.edu/static/public/45/data.csv")

In [14]:
raw_dataframes["Breast Cancer"] = pd.read_csv("https://archive.ics.uci.edu/static/public/15/data.csv")

In [15]:
raw_dataframes["Dermatology"] = pd.read_csv("https://archive.ics.uci.edu/static/public/33/data.csv")

In [16]:
raw_dataframes["Letters(E, F)"] = pd.read_csv("https://archive.ics.uci.edu/static/public/59/data.csv")

In [17]:
raw_dataframes["DNA"] = pd.read_csv("https://archive.ics.uci.edu/static/public/69/data.csv")

In [18]:
raw_dataframes["Mushroom"] = pd.read_csv("https://archive.ics.uci.edu/static/public/73/data.csv")

In [19]:
raw_dataframes["Iris"] = pd.read_csv("https://archive.ics.uci.edu/static/public/53/data.csv")

In [20]:
raw_dataframes["Isolet"] = pd.read_csv("isolet5.data", nrows=1560)

In [21]:
raw_dataframes["Optical"] = pd.read_csv("https://archive.ics.uci.edu/static/public/80/data.csv")

In [22]:
raw_dataframes["PenDigits"] = pd.read_csv("https://archive.ics.uci.edu/static/public/81/data.csv")

# Data preprocessing

In [23]:
# Dictionary to hold processed data
processed_dataframes = {}

# Process each dataframe in the raw_dataframes dictionary
for name, df in raw_dataframes.items():
    if name == "Letters(E, F)":
        y = df.iloc[:, 0]
        X = df.iloc[:, 1:]
    elif name == "Mushroom":
        y = df.iloc[:, 0]
        X = df.iloc[:, 1:]
    elif name == "DNA":
        y = df.iloc[:, 0].str.strip()
        # Remove the second column from the DNA dataframe
        X = df.iloc[:, 1:].drop(df.columns[1], axis=1)
    else:
        X, y = df.iloc[:, :-1], df.iloc[:, -1]

    # Limit rows for "Letters(E, F)" dataset
    if name == "Letters(E, F)" and len(X) > 1543:
        X = X.iloc[:1543]
        y = y.iloc[:1543]


    if name == "Isolet" and len(X) > 1560:
        X = X.iloc[:1560]
        y = y.iloc[:1560]
        
    # Drop columns with only 1 unique value
    for col in X.columns:
        if len(X[col].unique()) <= 1:
            X.drop(columns=[col], inplace=True)

    print(f"Before dropping rows with NaNs in {name}:")
    print(f"Shape of features (X): {X.shape}")
    print(f"Shape of targets (y): {y.shape}")
    print("NaN counts per column:\n", X.isnull().sum())

    # Drop rows that contain any NaN values
    X.dropna(axis=0, how='any', inplace=True)
    # After dropping NaNs, ensure y is aligned with X
    y = y[X.index]

    print(f"After dropping rows with NaNs in {name}:")
    print(f"Shape of features (X): {X.shape}")
    print(f"Shape of targets (y): {y.shape}")
    print("NaN counts per column:\n", X.isnull().sum())


    # Store processed data in processed_dataframes dictionary
    processed_dataframes[name] = {'features': X, 'targets': y}

Before dropping rows with NaNs in Soybean:
Shape of features (X): (47, 21)
Shape of targets (y): (47,)
NaN counts per column:
 date               0
plant-stand        0
precip             0
temp               0
hail               0
crop-hist          0
area-damaged       0
severity           0
seed-tmt           0
germination        0
leaves             0
lodging            0
stem-cankers       0
canker-lesion      0
fruiting-bodies    0
external-decay     0
mycelium           0
int-discolor       0
sclerotia          0
fruit-pods         0
roots              0
dtype: int64
After dropping rows with NaNs in Soybean:
Shape of features (X): (47, 21)
Shape of targets (y): (47,)
NaN counts per column:
 date               0
plant-stand        0
precip             0
temp               0
hail               0
crop-hist          0
area-damaged       0
severity           0
seed-tmt           0
germination        0
leaves             0
lodging            0
stem-cankers       0
canker-lesion      0

In [24]:
def encode_categorical(df, categorical_columns):
    """
    Encode non-numeric categorical columns to numeric categories.

    Args:
    df (pandas.DataFrame): DataFrame containing the data.
    categorical_columns (list of str): List of column names to encode.

    Returns:
    pandas.DataFrame: DataFrame with encoded columns.
    """
    label_encoders = {}
    for col in categorical_columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le  # Store encoder to allow inverse_transform if needed
    return df, label_encoders

In [25]:
def preprocess_datasets(dataframes):
    if 'Zoo' in dataframes:
        zoo_df = dataframes['Zoo']['features']
        if 0 in zoo_df.columns:
            zoo_df = zoo_df.drop(columns=[0])
        else:
            zoo_df = zoo_df.drop(columns=zoo_df.columns[0])
        dataframes['Zoo']['features'] = zoo_df

    if 'Heart Disease' in dataframes:
        hd_df = dataframes['Heart Disease']['features']
        columns_to_drop = [hd_df.columns[i] for i in [0, 3, 4, 7, 9] if i < len(hd_df.columns)]
        hd_df = hd_df.drop(columns=columns_to_drop)
        dataframes['Heart Disease']['features'] = hd_df
        y_hd = dataframes['Heart Disease']['targets']
        dataframes['Heart Disease']['targets'] = y_hd.apply(lambda x: 0 if x == 0 else 1)

    if 'Breast Cancer' in dataframes:
        bcw_df = dataframes['Breast Cancer']['features']
        bcw_df = bcw_df.drop(columns=bcw_df.columns[0])
        dataframes['Breast Cancer']['features'] = bcw_df

    if 'Dermatology' in dataframes:
        derm_df = dataframes['Dermatology']['features']
        derm_df = derm_df.drop(columns=derm_df.columns[-1])
        dataframes['Dermatology']['features'] = derm_df

    if 'Letters(E, F)' in dataframes:
        lr_ef_df = dataframes['Letters(E, F)']['features']
        lr_ef_targets = dataframes['Letters(E, F)']['targets']
        mask = lr_ef_targets.isin(['E', 'F'])
        dataframes['Letters(E, F)']['features'] = lr_ef_df[mask]
        dataframes['Letters(E, F)']['targets'] = lr_ef_targets[mask]

    if 'DNA' in dataframes:
        dna_df = dataframes['DNA']['features']
        # Encoding DNA sequences
        dna_df, _ = encode_categorical(dna_df, dna_df.columns.tolist())
        dataframes['DNA']['features'] = dna_df

    if 'Mushroom' in dataframes:
        mushroom_df = dataframes['Mushroom']['features']
        # Assuming all columns in Mushroom dataset are categorical and need encoding
        mushroom_df, _ = encode_categorical(mushroom_df, mushroom_df.columns.tolist())
        dataframes['Mushroom']['features'] = mushroom_df

    return dataframes

In [26]:
dataframes = preprocess_datasets(processed_dataframes)

# Clustering Ensemble on Numerical Datasets

In [27]:
def run_multiple_kmeans(features, n_clusters, n_runs):
    all_labels = []
    for i in range(n_runs):
        random_state = random.randint(0, 1000)
        kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=random_state, n_init=10)
        labels = kmeans.fit_predict(features)
        all_labels.append(labels)
    return np.array(all_labels).T

In [28]:
for dataset_name in datasets_ensemble:
    # Extracting features and targets from the preloaded datasets
    features = dataframes[dataset_name]["features"]
    targets = dataframes[dataset_name]["targets"]

    # Converting targets to numerical labels if they aren't already
    if targets.dtype.kind in 'O':  # Check if targets are object type (e.g., strings)
        targets = LabelEncoder().fit_transform(targets)

    # Determine the number of clusters from the unique elements in targets
    n_clusters = len(np.unique(targets))

    # Run multiple k-means and collect results
    ensembled_features = run_multiple_kmeans(features, n_clusters, num_runs)
    
    # Convert numpy array to DataFrame and replace the original data
    ensembled_features_df = pd.DataFrame(ensembled_features, index=features.index)
    dataframes[dataset_name]["features"] = ensembled_features_df

# Bernoulli Mixture Model

In [52]:
from scipy.special import logsumexp

class BernoulliMixture:
    def __init__(self, n_components, max_iter, tol=1e-3):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol

    def fit(self,x):
        self.x = x
        self.init_params()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.old_logL = self.get_log_likelihood(log_bernoullis)
        for step in range(self.max_iter):
            if step > 0:
                self.old_logL = self.logL
            # E-Step
            self.gamma = self.get_responsibilities(log_bernoullis)
            self.remember_params()
            # M-Step
            self.get_Neff()
            self.get_mu()
            self.get_pi()
            # Compute new log_likelihood:
            log_bernoullis = self.get_log_bernoullis(self.x)
            self.logL = self.get_log_likelihood(log_bernoullis)
            if np.isnan(self.logL):
                self.reset_params()
                break

    def reset_params(self):
        self.mu = self.old_mu.copy()
        self.pi = self.old_pi.copy()
        self.gamma = self.old_gamma.copy()
        self.get_Neff()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)

    def remember_params(self):
        self.old_mu = self.mu.copy()
        self.old_pi = self.pi.copy()
        self.old_gamma = self.gamma.copy()

    def init_params(self):
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        self.pi = 1/self.n_components * np.ones(self.n_components)
        self.mu = np.random.RandomState(seed=0).uniform(low=0.25, high=0.75, size=(self.n_components, self.n_features))
        self.normalize_mu()

    def normalize_mu(self):
        sum_over_features = np.sum(self.mu, axis=1)
        for k in range(self.n_components):
            self.mu[k,:] /= sum_over_features[k]

    def get_responsibilities(self, log_bernoullis):
        gamma = np.zeros(shape=(log_bernoullis.shape[0], self.n_components))
        Z =  logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
        for k in range(self.n_components):
            gamma[:, k] = np.exp(np.log(self.pi[k]) + log_bernoullis[:,k] - Z)
        return gamma

    def get_log_bernoullis(self, x):
        log_bernoullis = self.get_save_single(x, self.mu)
        log_bernoullis += self.get_save_single(1-x, 1-self.mu)
        return log_bernoullis

    # Modified
    def get_save_single(self, x, mu):
        # Ensure x and mu are numpy arrays with float type
        x = np.array(x, dtype=np.float64)
        mu = np.array(mu, dtype=np.float64)

        # Avoid taking log of zero by setting a minimum value
        mu_place = np.where(mu <= 1e-15, 1e-15, mu)
        # Perform the tensor dot product safely
        try:
            result = np.tensordot(x, np.log(mu_place), axes=(1, 1))
        except TypeError as e:
            print("TypeError encountered:", e)
            print("x shape:", x.shape, "x dtype:", x.dtype)
            print("mu_place shape:", mu_place.shape, "mu_place dtype:", mu_place.dtype)
            raise

        return result

    def get_Neff(self):
        self.Neff = np.sum(self.gamma, axis=0)

    def get_mu(self):
        self.mu = np.einsum('ik,id -> kd', self.gamma, self.x) / self.Neff[:,None] 

    def get_pi(self):
        self.pi = self.Neff / self.n_samples

    def predict(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)

    def get_sample_log_likelihood(self, log_bernoullis):
        return logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)

    def get_log_likelihood(self, log_bernoullis):
        return np.mean(self.get_sample_log_likelihood(log_bernoullis))

    def score(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_log_likelihood(log_bernoullis)

    def score_samples(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_sample_log_likelihood(log_bernoullis)

# Schotastic Bernoulli Mixture Model

In [79]:
from scipy.special import logsumexp

class StochasticBernoulliMixture:

    def __init__(self, n_components, max_iter, tol=1e-3, batch_size=100):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
        self.batch_size = batch_size  # Added line: Initialize batch size

    def fit(self, x):
        self.x = x
        self.init_params()
        n_samples = x.shape[0]
        batch_size = min(self.batch_size, n_samples)

        for step in range(self.max_iter):
            indices = np.random.choice(n_samples, batch_size, replace=False)
            x_batch = x[indices]

            if step > 0:
                self.old_logL = self.logL

            log_bernoullis = self.get_log_bernoullis(x_batch)
            self.gamma = self.get_responsibilities(log_bernoullis)
            self.remember_params()

            self.get_Neff()
            self.get_mu(x_batch)
            self.get_pi()

            log_bernoullis = self.get_log_bernoullis(x)
            self.logL = self.get_log_likelihood(log_bernoullis)

            if np.isnan(self.logL):
                self.reset_params()
                break

    def reset_params(self):
        self.mu = self.old_mu.copy()
        self.pi = self.old_pi.copy()
        self.gamma = self.old_gamma.copy()
        self.get_Neff()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)

    def remember_params(self):
        self.old_mu = self.mu.copy()
        self.old_pi = self.pi.copy()
        self.old_gamma = self.gamma.copy()

    def init_params(self):
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        self.pi = 1/self.n_components * np.ones(self.n_components)
        self.mu = np.random.RandomState(seed=0).uniform(low=0.25, high=0.75, size=(self.n_components, self.n_features))
        self.normalize_mu()

    def normalize_mu(self):
        sum_over_features = np.sum(self.mu, axis=1)
        for k in range(self.n_components):
            self.mu[k,:] /= sum_over_features[k]

    def get_responsibilities(self, log_bernoullis):
        gamma = np.zeros(shape=(log_bernoullis.shape[0], self.n_components))
        Z =  logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
        for k in range(self.n_components):
            gamma[:, k] = np.exp(np.log(self.pi[k]) + log_bernoullis[:,k] - Z)
        return gamma

    def get_log_bernoullis(self, x):
        log_bernoullis = self.get_save_single(x, self.mu)
        log_bernoullis += self.get_save_single(1-x, 1-self.mu)
        return log_bernoullis

    # Modified
    def get_save_single(self, x, mu):
        # Ensure x and mu are numpy arrays with float type
        x = np.array(x, dtype=np.float64)
        mu = np.array(mu, dtype=np.float64)

        # Avoid taking log of zero by setting a minimum value
        mu_place = np.where(mu <= 1e-15, 1e-15, mu)
        # Perform the tensor dot product safely
        try:
            result = np.tensordot(x, np.log(mu_place), axes=(1, 1))
        except TypeError as e:
            print("TypeError encountered:", e)
            print("x shape:", x.shape, "x dtype:", x.dtype)
            print("mu_place shape:", mu_place.shape, "mu_place dtype:", mu_place.dtype)
            raise

        return result

    def get_Neff(self):
        self.Neff = np.sum(self.gamma, axis=0)

    def get_mu(self, x):
        self.mu = np.einsum('ik,id -> kd', self.gamma, x) / self.Neff[:, None]

    def get_pi(self):
        self.pi = self.Neff / self.n_samples

    def predict(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)

    def get_sample_log_likelihood(self, log_bernoullis):
        return logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)

    def get_log_likelihood(self, log_bernoullis):
        return np.mean(self.get_sample_log_likelihood(log_bernoullis))

    def score(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_log_likelihood(log_bernoullis)

    def score_samples(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_sample_log_likelihood(log_bernoullis)

# Kmodes

In [57]:
def perform_kmodes(features, n_clusters):
    """Perform clustering using KModes algorithm."""
    km = KModes(n_clusters=n_clusters, init='random', n_init=5)
    clusters = km.fit_predict(features)
    return clusters

# Running the Schotastic Bernoulli Mixture Model, Bernoulli Mixture Model, Kmodes

In [58]:
def preprocess_data(features):
    if 'sequence' in features.columns:
        # Assuming DNA sequences are stored in 'sequence' column
        features['encoded_sequence'] = encode_dna_sequences(features['sequence'])
        features.drop('sequence', axis=1, inplace=True)  # Remove the original sequence column
        return features['encoded_sequence']
    return features

In [59]:
def calculate_metrics(true_labels, predicted_labels):
    """
    Calculate clustering metrics: Adjusted Rand Index, Normalized Mutual Information, and Fowlkes-Mallows Index.
    
    Args:
    true_labels (array-like): True cluster labels.
    predicted_labels (array-like): Cluster labels predicted by a clustering algorithm.
    
    Returns:
    tuple: A tuple containing the ARI, NMI, and FMI scores.
    """
    ari_score = ARI(true_labels, predicted_labels)
    nmi_score = NMI(true_labels, predicted_labels)
    fmi_score = FMI(true_labels, predicted_labels)
    return ari_score, nmi_score, fmi_score

In [76]:
import numpy as np
import pandas as pd

def run_clustering_algorithms(dataframes, n_clusters_dict, num_runs=10):
    results_list = []
    for name, data in dataframes.items():
        print("Processing:", name)
        features = data['features']
        true_labels = data['targets'].squeeze()
        n_clusters = n_clusters_dict.get(name, 2)

        metrics = {'KModes': [], 'BernoulliMixture': [], 'StochasticBernoulliMixture': []}

        features_encoded = OneHotEncoder(sparse_output=False).fit_transform(features)

        for _ in range(num_runs):
            # K-Modes
            km_clusters = perform_kmodes(features, n_clusters)
            ari, nmi, fmi = calculate_metrics(true_labels, km_clusters)
            metrics['KModes'].append((ari, nmi, fmi))

            # Original Bernoulli Mixture
            bm = BernoulliMixture(n_components=n_clusters, max_iter=1000)
            bm.fit(features_encoded)
            bm_clusters = bm.predict(features_encoded)
            ari, nmi, fmi = calculate_metrics(true_labels, bm_clusters)
            metrics['BernoulliMixture'].append((ari, nmi, fmi))

            # Stochastic Bernoulli Mixture
            sbm = StochasticBernoulliMixture(n_components=n_clusters, max_iter=1000, batch_size=100)
            sbm.fit(features_encoded)
            sbm_clusters = sbm.predict(features_encoded)
            ari, nmi, fmi = calculate_metrics(true_labels, sbm_clusters)
            metrics['StochasticBernoulliMixture'].append((ari, nmi, fmi))

        # Calculate mean and standard deviation for each method and append to results list
        for method, values in metrics.items():
            ari_vals, nmi_vals, fmi_vals = zip(*values)  # Unpack the metrics values
            ari_mean, ari_std = np.mean(ari_vals), np.std(ari_vals)
            nmi_mean, nmi_std = np.mean(nmi_vals), np.std(nmi_vals)
            fmi_mean, fmi_std = np.mean(fmi_vals), np.std(fmi_vals)  # Calculate mean and std for FMI
            results_list.append({
                "Dataset": name,
                "Method": method,
                "ARI": f"{ari_mean:.4f}±{ari_std:.2f}",
                "NMI": f"{nmi_mean:.4f}±{nmi_std:.2f}",
                "FMI": f"{fmi_mean:.4f}±{fmi_std:.2f}"  # Include FMI in results
            })

    # Convert list of dictionaries to DataFrame for results
    results_df = pd.DataFrame(results_list)
    return results_df


In [77]:
def reformat_results(results_df):
    # Include 'FMI' in the list of columns to be melted along with 'ARI' and 'NMI'
    expanded_df = pd.melt(results_df, id_vars=["Dataset", "Method"], value_vars=["ARI", "NMI", "FMI"], var_name="Metric", value_name="Value")
    
    # Split the 'Value' column into 'Metric_Value' and 'Std', assuming the format "value±std"
    expanded_df[['Metric_Value', 'Std']] = expanded_df['Value'].str.split('±', expand=True)
    expanded_df.drop(columns=['Value'], inplace=True)  # Removing the original combined column
    
    # Convert the 'Metric_Value' and 'Std' columns to numeric types for further operations
    expanded_df['Metric_Value'] = expanded_df['Metric_Value'].astype(float)
    expanded_df['Std'] = expanded_df['Std'].astype(float)

    # Reformat 'Metric_Value' to combine mean and standard deviation into a single string formatted as needed
    expanded_df['Metric_Value'] = expanded_df['Metric_Value'].map('{:.4f}'.format) + "±" + expanded_df['Std'].map('{:.2f}'.format)
    
    # Ensuring the order of datasets and methods remains consistent with the original DataFrame
    dataset_order = results_df['Dataset'].unique()
    method_order = results_df['Method'].unique()

    # Creating a pivot table to restructure the DataFrame
    # This pivot table organizes the data by 'Dataset' and 'Metric', with methods as columns and metric values as cell data
    pivot_df = expanded_df.pivot_table(index=["Dataset", "Metric"], columns="Method", values="Metric_Value", aggfunc='first')
    
    # Reindexing the pivot table to maintain the original order
    pivot_df = pivot_df.reindex(dataset_order, level='Dataset')
    pivot_df = pivot_df.reindex(method_order, axis='columns')

    return pivot_df

In [80]:
# Running all algorithms and storing the results
results = run_clustering_algorithms(dataframes, num_clusters_dict, num_runs)

Processing: Soybean
Processing: Zoo
Processing: Heart Disease
Processing: Breast Cancer
Processing: Dermatology
Processing: Letters(E, F)
Processing: DNA
Processing: Mushroom
Processing: Iris
Processing: Isolet
Processing: Optical
Processing: PenDigits


In [81]:
# Use the function to reformat the results
formatted_results = reformat_results(results)

# Presentation of Results


In [82]:
# Prepare data for the table
table_data = []

for name, content in dataframes.items():
    X, y = content['features'], content['targets']
    table_data.append({
        "Name": name,
        "Number of Samples": X.shape[0],
        "Number of Features": X.shape[1],
        "Number of Unique Values in Target": len(pd.unique(y))
    })

# Convert table data into a DataFrame for pretty printing
table_df = pd.DataFrame(table_data)
table_df

,Name,Number of Samples,Number of Features,Number of Unique Values in Target
0,Soybean,47,21,4
1,Zoo,101,16,7
2,Heart Disease,297,8,2
3,Breast Cancer,683,9,2
4,Dermatology,358,33,6
5,"Letters(E, F)",103,16,2
6,DNA,3190,60,3
7,Mushroom,5644,21,6
8,Iris,150,10,3
9,Isolet,1558,10,26


In [83]:
formatted_results

Method                      KModes BernoulliMixture StochasticBernoulliMixture
Dataset       Metric                                                          
Soybean       ARI      0.8799±0.15      1.0000±0.00                1.0000±0.00
              FMI      0.9137±0.11      1.0000±0.00                1.0000±0.00
              NMI      0.9253±0.09      1.0000±0.00                1.0000±0.00
Zoo           ARI      0.7000±0.14      0.6972±0.00                0.7267±0.00
              FMI      0.7658±0.11      0.7645±0.00                0.7882±0.00
              NMI      0.7893±0.05      0.8028±0.00                0.8202±0.00
Heart Disease ARI      0.3406±0.04      0.3292±0.00                0.1628±0.06
              FMI      0.6739±0.02      0.6646±0.00                0.6151±0.01
              NMI      0.2684±0.02      0.2598±0.00                0.1900±0.03
Breast Cancer ARI      0.7438±0.05      0.8800±0.00                0.0497±0.02
              FMI      0.8876±0.02      0.9445±0.00                0.5918±0.00
              NMI      0.6397±0.04      0.8152±0.00                0.2390±0.01
Dermatology   ARI      0.5235±0.12      0.7718±0.00                0.7324±0.11
              FMI      0.6228±0.10      0.8188±0.00                0.8031±0.07
              NMI      0.6304±0.07      0.8291±0.00                0.7825±0.07
Letters(E, F) ARI      0.3899±0.08      0.0117±0.00                0.0031±0.01
              FMI      0.7029±0.03      0.5106±0.00                0.5739±0.04
              NMI      0.3303±0.06      0.0135±0.00                0.0132±0.02
DNA           ARI      0.0279±0.01      0.8589±0.00                0.0492±0.07
              FMI      0.3772±0.00      0.9132±0.00                0.6115±0.02
              NMI      0.0430±0.01      0.7779±0.00                0.0783±0.08
Mushroom      ARI     -0.0139±0.00     -0.0005±0.00               -0.0181±0.00
              FMI      0.5230±0.00      0.4747±0.00                0.4948±0.00
              NMI      0.0202±0.00      0.0427±0.00                0.0342±0.00
Iris          ARI      0.7302±0.00      0.7302±0.00                0.7302±0.00
              FMI      0.8208±0.00      0.8208±0.00                0.8208±0.00
              NMI      0.7582±0.00      0.7582±0.00                0.7582±0.00
Isolet        ARI      0.4381±0.02      0.4122±0.00                0.2884±0.07
              FMI      0.4640±0.02      0.4404±0.00                0.3612±0.05
              NMI      0.7105±0.01      0.7003±0.00                0.6376±0.03
Optical       ARI      0.6711±0.00      0.6760±0.00                0.5894±0.07
              FMI      0.7070±0.00      0.7145±0.00                0.6504±0.05
              NMI      0.7569±0.00      0.7628±0.00                0.7290±0.02
PenDigits     ARI      0.5367±0.02      0.5320±0.00                0.4895±0.05
              FMI      0.5916±0.02      0.5875±0.00                0.5609±0.04
              NMI      0.6820±0.01      0.6817±0.00                0.6506±0.04

# Analysis Report

### Why FMI performs better than NMI and ARI

The Fowlkes-Mallows Index (FMI) generally tends to be higher because it is less stringent regarding error types—it emphasizes correctly identified pairs rather than penalizing incorrectly separated or grouped pairs. In contrast, the Adjusted Rand Index (ARI) and Normalized Mutual Information (NMI) more rigorously account for both false positives and false negatives, leading to typically lower scores.

### Why FMI is consistently higher than ARI and NMI:

FMI measures the geometric mean of the proportion of pairs correctly identified together (same cluster in both predicted and true labels) and the proportion of pairs in the same cluster in at least one of the predicted or true labels. It tends to emphasize true positives more heavily, resulting in consistently higher scores across all datasets.

ARI, however, measures the similarity between two assignments while discounting pairs that are randomly assigned to the same or different clusters. This index accounts for chance pairings, making its scores generally lower than FMI, especially if many pairs are randomly classified.

NMI is a normalized version of the Mutual Information (MI) score that evaluates the agreement between true and predicted labels while considering cluster randomness. Its scores typically fall between those of ARI and FMI.


## Conclusion on the different Models: Kmodes, Bernoulli Mixter, Stochastic Bernoulli Mixture
The superior performance of Bernoulli-based methods on the Soybean dataset indicates a good fit for datasets with binary or nearly binary data representations.

KModes generally performs consistently across various data types but does not excel when data aligns better with binary assumptions.

The performance variation across different datasets and metrics for the Stochastic Bernoulli Mixture model suggests that while randomness can sometimes better capture data variability, it can also destabilize the clustering process, particularly when the underlying data structure is complex or not well-understood.